In [1]:
# https://www.geeksforgeeks.org/scrape-linkedin-using-selenium-and-beautiful-soup-in-python/

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import time
import json

## Logging in to LinkedIn

In [3]:
# Creating a webdriver instance
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.46M/6.46M [00:00<00:00, 11.7MB/s]
C:\Users\Joyce\AppData\Local\Temp\ipykernel_14068\2930654737.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")
# waiting for the page to load
time.sleep(2)

In [5]:
# entering username
username = driver.find_element(By.ID, "username")
# Enter Your Email Address
username.send_keys("sh2429@cornell.edu")

In [6]:
# entering password
pword = driver.find_element(By.ID, "password")
# Enter Your Password
pword.send_keys("qzwx.1506")

In [7]:
# Clicking on the log in button
# Format (syntax) of writing XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

## Extracting Job Search Data

In [8]:
# Opening job's page
# paste the URL of job's page here
job_url = "https://www.linkedin.com/jobs/"
# this will open the link
driver.get(job_url)
time.sleep(2)

In [9]:
# entering search keywords
search = driver.find_element(By.XPATH, "//input[starts-with(@id, 'jobs-search-box-keyword-id-ember')]")
search.send_keys("software engineer")
time.sleep(2)
search.send_keys(Keys.RETURN)
time.sleep(2)

In [10]:
# scroll
def scroll():
    print("start scrolling", end = ' ')
    left_panel = driver.find_element(By.CLASS_NAME, "jobs-search-results-list")
    time.sleep(1)
    verical_ordinate = 100
    for i in range(0, 10):
        print(".", end = ' ')
        driver.execute_script("arguments[0].scrollTop = arguments[1]", left_panel, verical_ordinate)
        verical_ordinate += 1000
        time.sleep(1)
    print("done scrolling")

In [11]:
# process jobs on one page
def process_page():
    job_src = driver.page_source
    soup = BeautifulSoup(job_src, 'lxml')
    # get the list of job titles
    jobs_html = soup.find_all('a', {'class': 'job-card-list__title'})

    # loop through jobs on this page
    for title in jobs_html:
        try:
            job_data = {}
            print("processing position: " + title.text.strip())
            job_data["position"] = title.text.strip()

            # click on each job to get detailed information
            driver.find_element(By.LINK_TEXT, title.text.strip()).click()
            time.sleep(2)
            detail_src = driver.page_source
            soup = BeautifulSoup(detail_src, 'lxml')
            detail = soup.find('section', {'class': 'scaffold-layout__detail'})

            # get job id
            url = driver.current_url
            index_start = url.find('currentJobId=') + 13
            index_end = url.find('&keywords=')
            info_id = url[index_start:index_end] if (index_start != -1 and index_end != -1) else ""
            job_data["job_id"] = info_id

            # get company name
            info_company = soup.find('span', {'class': 'jobs-unified-top-card__company-name'})
            job_data["company"] = info_company.text.strip() if info_company is not None else ""

            # get date posted
            info_date = soup.find('span', {'class': 'jobs-unified-top-card__posted-date'})
            job_data["date"] = info_date.text.strip() if info_date is not None else ""

            # get type and level
            info_type = soup.find('li', {'class': 'jobs-unified-top-card__job-insight'})
            job_data["type"] = info_type.text.strip() if info_type is not None else ""

            # get description
            info_description = soup.find('div', {'class': 'jobs-description-content'})
            job_data["description"] = info_description.text.strip() if info_description is not None else ""
            data.append(job_data)
        except:
            print("something wrong here :(")

In [12]:
# loop through the pages
page_lst = driver.find_element(By.CLASS_NAME, "artdeco-pagination__pages")
pages = page_lst.find_elements(By.TAG_NAME, "li")
start = int(pages[0].text)
end = int(pages[-1].text)
data = []

for page in range(start, end+1):
    driver.find_element(By.XPATH, "//button[@aria-label='Page " + str(page) + "']").click()
    time.sleep(2)
    print("currently on page: " + str(page))
    try:
        scroll()
        process_page()
        print("================================================")
    except:
        print("something wrong here :(")

currently on page: 1
start scrolling . . . . . . . . . . done scrolling
processing position: Software Engineer
processing position: Firmware Engineer
processing position: Full Stack Software Engineer
processing position: Junior Software Engineer (RF Engineering)
processing position: Software Engineer Intern- Data Path
processing position: Junior Software Developer
processing position: Cloud Security Engineer I (M365/O365)
processing position: Web Application Developer (IT)
processing position: Junior Data Engineer
processing position: Software Engineer (Entry-Level)- Herndon, VA
processing position: Software Developer (Junior)
processing position: Senior Data Engineer
processing position: Azure Data Engineer
processing position: Machine Learning Engineer - Intern
processing position: Software Engineer III
processing position: Software Engineer
processing position: Entry Level Software Engineer
processing position: Software Engineering Intern
processing position: 2023 Intern: Research S

processing position: Embedded Software Engineer
processing position: ServiceNow Developer
processing position: Entry Level software Engineer
processing position: Machine Learning Engineer
processing position: Front End Engineer, Hybrid-Remote
processing position: Software Engineer
processing position: Sr React Developer
processing position: Staff Cloud Engineer
processing position: Software Engineer
processing position: Jr. Software Engineer-Power Platform
processing position: Back End Software Engineer - Mid Level
processing position: Back End Software Engineer - Mid level
processing position: Software Developer
processing position: Entry Level Software Engineer
currently on page: 8
start scrolling . . . . . . . . . . done scrolling
processing position: Senior Mainframe Engineer
processing position: Hiring!! React.Js Developer with Chat.Js
processing position: Front End Software Engineer - Junior Level
processing position: Data Modeler Engineer
processing position: Front End Software 

processing position: Software Engineer
currently on page: 14
start scrolling . . . . . . . . . . done scrolling
processing position: Principal Android Developer
processing position: Systems Engineer 3 - Contractor - Remote work possible
processing position: Linux Engineer
processing position: Python Developer
processing position: Fullstack Software Engineer (100% remote)
processing position: Fullstack Software Engineer (100% remote)
processing position: Programmer
processing position: Spring 2023: Floorcare Software Engineering Intern (Undergraduate)
processing position: Jr .NET Software Developer
processing position: AWS Cloud Data Engineer
processing position: Sr. Firmware Engineer - Camera
processing position: Fullstack Software Engineer (100% remote)
processing position: Software Dev Engineer ||
processing position: Senior Software Engineer, Back End (Hybrid)
processing position: E-Trading Python Software Developer
processing position: Staff Software Engineer - Frontend
processing 

processing position: Software Development Engineer I
processing position: Python Developer
processing position: Software Engineer - Internship (2023)
processing position: Data Analytics Engineer | US CITIZENSHIP
processing position: Software Developer
processing position: Python Developer
processing position: Software Engineer , Prime Air
processing position: Software Engineer
processing position: Software Engineer
processing position: Principal Software Engineer
processing position: Fullstack Software Engineer (100% remote)
processing position: Software Developer- Early Career
processing position: Software Engineer - Early Career
processing position: Android Software Engineer, TikTok-Social
processing position: Software Engineer
processing position: Staff Software Engineer, Engineering Productivity, Google Cloud Platform
processing position: Software Engineer
processing position: Software Engineer
processing position: Security Software Engineer Intern 2023
currently on page: 21
start 

processing position: Fullstack Software Engineer, TikTok Open Platform
processing position: Security Engineer
processing position: Software Engineer, Games Co-Op
processing position: Software Engineer
processing position: Software Developer, Full-time
processing position: Model Based Systems Engineer
processing position: (GW) Associate Machine Learning Engineer
processing position: Software Engineer Staff
processing position: Scientific Software Developer
processing position: Software Engineer III
processing position: 100% Remote Android Developer
processing position: Java Software Engineer
processing position: Junior Level Systems Engineer
processing position: Embedded Software Engineer
currently on page: 27
start scrolling . . . . . . . . . . done scrolling
processing position: DevOps/Azure Cloud Engineer - Terraform, PowerShell - Austin, TX 37759
processing position: Database Engineer - Mid Level
processing position: IA Security Engineer Sr
processing position: Linux/VMware Engineer

processing position: Senior Systems Engineer
processing position: Security Software Engineer Intern 2023
processing position: Software/Data Engineer (PYTHON OR SPARK) HYBRID WORK MODEL
processing position: Java Developer-Location Open
processing position: Application Developer: Maximo
processing position: Cobol Programmer
processing position: Backend Engineer
processing position: Security Software Engineer Intern 2023
processing position: Senior Software Engineer
processing position: Software Engineer - Client Reporting
processing position: Java Developer
processing position: Software Engineer/.NET Developer
processing position: Associate Software Engineer/Software Engineer(1/2)
currently on page: 33
start scrolling . . . . . . . . . . done scrolling
processing position: Data Normalization Systems Engineer
processing position: Senior Mainframe Developer - Returner to Work
processing position: Senior Software Engineer- Clothing Company
processing position: Sr. Front End Engineer
process

currently on page: 39
start scrolling . . . . . . . . . . done scrolling
processing position: Remote Micro Focus Cobol Developer
processing position: Chief Infrastructure Engineer
processing position: Software Engineering Intern
processing position: Staff Software Engineer
processing position: Java Developer
processing position: Java Developer
processing position: Full Stack Observability Engineer - Splunk
processing position: Data Engineer (Remote 361 days, 4 days on-site)
processing position: Java Developer(Jr/Sr/Lead)-100% Remote
processing position: Angular Front End Developer (Barcelona, Full remote)
processing position: Software Engineer
processing position: Backend Java Developer
processing position: Full Stack Software Developer Remote
processing position: Software Engineer II - C/C++/OS
processing position: Software Engineer II - C/C++/OS
processing position: Full Stack Developer
processing position: Java Backend Developer
processing position: Software Engineer In Test
process

In [13]:
json_object = json.dumps(data)
with open(("LinkedIn" + datetime.today().strftime('%Y%m%d%H%M') + ".json"), "w") as outfile:
    outfile.write(json_object)